In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
import re
def delnums(my_str):
    return re.sub('[0-9]+', '', my_str)

In [3]:
df = gpd.read_file('countylist.csv')

In [4]:
df = df[df.state == 'AL']
df

,id,state,FIPS,county,countyseat,population,landarea_imp,landarea_si,waterarea_imp,waterarea_si,totalarea_imp,totalarea_si,lat,lon,geometry
0,1,AL,01001,Autauga,Prattville,54571,1539.582,594.436,25.776,9.952,1565.358,604.388,+32.536382,-86.644490,None
1,2,AL,01003,Baldwin,Bay Minette,182265,4117.522,1589.784,1133.190,437.527,5250.712,2027.311,+30.659218,-87.746067,None
2,3,AL,01005,Barbour,Clayton,27457,2291.819,884.876,50.865,19.639,2342.684,904.515,+31.870670,-85.405456,None
3,4,AL,01007,Bibb,Centreville,22915,1612.481,622.582,9.289,3.587,1621.770,626.169,+33.015893,-87.127148,None
4,5,AL,01009,Blount,Oneonta,57322,1669.962,644.776,15.157,5.852,1685.119,650.628,+33.977448,-86.567246,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,63,AL,01125,Tuscaloosa,Tuscaloosa,194656,3423.329,1321.755,76.396,29.497,3499.725,1351.252,+33.290202,-87.522860,None
63,64,AL,01127,Walker,Jasper,67023,2049.177,791.192,36.264,14.002,2085.441,805.194,+33.791571,-87.301092,None
64,65,AL,01129,Washington,Chatom,17581,2797.722,1080.207,21.651,8.359,2819.373,1088.566,+31.406974,-88.202078,None
65,66,AL,01131,Wilcox,Camden,11670,2301.200,888.498,48.985,18.913,2350.185,907.411,+31.990330,-87.302205,None


In [5]:
import tabula

In [6]:
file = "Champion_Trees.pdf"

table = tabula.read_pdf(file,pages=3)
df = table[0]
names = df.loc[1::3]['Unnamed: 0'].values
primary_name = []
secondary_name = []
for i, s in enumerate(names):
    newstr = s.split(',')
    primary_name.append(newstr[0])
    if len(newstr) == 2:
        secondary_name.append(newstr[1])
    else:
        secondary_name.append('N/A')


Got stderr: Apr 18, 2021 11:28:27 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode



In [7]:
arr = df.loc[2::3]['Unnamed: 0']
scientific_name = []
Year_Named = []
for i, s in enumerate(arr):
    try:
        Year_Named.append(int(s[-4:]))
    except:
        Year_Named.append(np.nan)
    newstr = delnums(s)
    scientific_name.append(newstr.strip())


In [8]:
circumference = df.loc[2::3]['(inches)'].values
height = df.loc[2::3]['(feet)'].values
spread = df.loc[2::3]['(feet).1'].values
total_points = df.loc[2::3]['Points'].values
county =  df.loc[2::3]['Unnamed: 1'].values
Year_remeasured = df.loc[2::3]['Year'].astype(int).values


In [9]:
arr = df.loc[2::3]['Nominator (N)'].values
owner = []
nominator = []
for i, s in enumerate(arr):
    s = s.replace('–',',')
    s = s.replace('-',',')
    #if '-O' in s: s=s.replace('-O',' - O')
    newlist = s.split(',')
    owner.append(newlist[0].strip())
    if s.strip()[-1] == 'N':
        nominator.append(newlist[0].strip())
    else:
        nominator.append('TBD')

In [10]:
arr = df.loc[3::3]['Nominator (N)'].reset_index().drop('index',axis=1)
for i in np.where(arr.notnull())[0]:
    s = arr['Nominator (N)'][i]
    s = s.replace('–',',')
    s = s.replace('-',',')
    newstr = s.split(',')
    nominator[i] = newstr[0].strip()

In [11]:
mdf = pd.DataFrame(
{
    'species':primary_name,
    'subspecies':secondary_name,
    'scientific_name':scientific_name,
    'year_named':Year_Named,
    'circumference_in':circumference,
    'height_in':height,
    'spread_ft':spread,
    'total_points':total_points,
    'county':county,
    'year_remeasured':Year_remeasured,
    'owner':owner,
    'nominator':nominator
})

In [12]:
mdf

,species,subspecies,scientific_name,year_named,circumference_in,height_in,spread_ft,total_points,county,year_remeasured,owner,nominator
0,ALDER,Hazel,"Alnus serrulata,",2013,15.0,36.0,18.0,56.0,Talladega,2018,Allen McBride,John McBride
1,AMERICAN SMOKETREE,N/A,"Cotinus obovatu,",2010,61.0,64.0,38.0,135.0,Madison,2021,Carol Rampey,Melody Stewart
2,APPLE,Southern crab,"Malus angustifolia,",2002,59.0,39.0,27.0,105.0,Montgomery,2017,Silas Bonner,Brent Gohring
3,ASH,Blue,"Fraxinus quadrangulata,",2012,84.0,80.0,41.0,174.0,Jackson,2017,National Park Service,Jason Shelton
4,ASH,Green,"Fraxinus pennsylvanica,",2000,177.0,88.0,84.0,286.0,Wilcox,2018,Allyrae Wallace Educational Trust,Jesse Bell
5,ASH,White,"Fraxinus americana,",2020,156.0,74.0,92.0,253.0,Shelby,2025,Clay Holland,Charles Ford
6,BALDCYPRESS,N/A,"Taxodium distichum,",1989,326.0,112.0,56.0,452.0,Baldwin,2016,AL Dep’t. of Conservation,Michael Jordon & John Martin
7,BASSWOOD,N/A,"Tilia spp.,",2016,131.0,115.0,52.0,259.0,Madison,2021,City of Huntsville,Susan Weber
8,BAYBERRY,Odorless,"Morella inodora,",2000,15.0,27.0,16.0,46.0,Baldwin,2016,City of Daphne,Dr. Harry Larsen & Fred Nation
9,BEECH,American,"Fagus grandifolia,",2015,114.0,131.0,74.0,264.0,Autauga,2020,City of Prattville,Deven Peek


In [13]:
file = "Champion_Trees.pdf"


for p in range(4,11):
    print('Page ' + str(p))
    table = tabula.read_pdf(file,pages=p)
    df = table[0]

    names = df.loc[1::3]['Unnamed: 0'].values
    primary_name = []
    secondary_name = []
    for i, s in enumerate(names):
        newstr = s.split(',')
        primary_name.append(newstr[0])
        if len(newstr) == 2:
            secondary_name.append(newstr[1])
        else:
            secondary_name.append('N/A')
    arr = df.loc[2::3]['Unnamed: 0']
    scientific_name = []
    Year_Named = []
    for i, s in enumerate(arr):
        try:
            Year_Named.append(int(s[-4:]))
        except:
            Year_Named.append(np.nan)
        newstr = delnums(s)
        scientific_name.append(newstr.strip())

        
        
        
    circumference = df.loc[2::3]['(inches)'].values
    height = df.loc[2::3]['(feet)'].values
    spread = df.loc[2::3]['(feet).1'].values
    total_points = df.loc[2::3]['Points'].values
    county =  df.loc[2::3]['Unnamed: 1'].values
    Year_remeasured = df.loc[2::3]['Year'].astype(int).values

    arr = df.loc[2::3]['Nominator (N)'].values
    owner = []
    nominator = []
    for i, s in enumerate(arr):
        s = s.replace('–',',')
        s = s.replace('-',',')
        #if '-O' in s: s=s.replace('-O',' - O')
        newlist = s.split(',')
        owner.append(newlist[0].strip())
        if s.strip()[-1] == 'N':
            nominator.append(newlist[0].strip())
        else:
            nominator.append('TBD')
    arr = df.loc[3::3]['Nominator (N)'].reset_index().drop('index',axis=1)        
    for i in np.where(arr.notnull())[0]:
        s = arr['Nominator (N)'][i]
        s = s.replace('–','-')
        newstr = s.split(' - ')
        nominator[i] = newstr[0]
    newdf = pd.DataFrame(
    {
        'species':primary_name,
        'subspecies':secondary_name,
        'scientific_name':scientific_name,
        'year_named':Year_Named,
        'circumference_in':circumference,
        'height_in':height,
        'spread_ft':spread,
        'total_points':total_points,
        'county':county,
        'year_remeasured':Year_remeasured,
        'owner':owner,
        'nominator':nominator
    })    
    mdf = pd.concat((newdf,mdf))

Page 4


Got stderr: Apr 18, 2021 11:28:29 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode



Page 5


Got stderr: Apr 18, 2021 11:28:30 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode



Page 6
Page 7


Got stderr: Apr 18, 2021 11:28:32 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode



Page 8
Page 9
Page 10


In [65]:
mdf = mdf.reset_index().drop('index',axis=1)
mdf['scientific_name'] = [i.replace(',','') for i in mdf.scientific_name.values]
mdf['sci_lc'] = [i.lower().strip() for i in mdf.scientific_name.values]
len(np.unique(np.array(mdf.sci_lc.values)))

141

In [15]:
import wikipedia

possible_entries = wikipedia.search("BUTTERNUT")

In [16]:
grp = mdf.groupby('scientific_name')

In [63]:
grp.scientific_name.count()

sci_lc
quercus shumardii    1
Name: scientific_name, dtype: int64

In [62]:
s

1

In [335]:
summaries

['Alder is the common name of a genus of flowering plants, Alnus, belonging to the birch family Betulaceae. The genus comprises about 35 species of monoecious trees and shrubs, a few reaching a large size, distributed throughout the north temperate zone with a few species extending into Central America, as well as the northern and southern Andes.\n\n',
 '"Smoke tree" may refer to any of several plants, some of whose parts are finely divided and give the appearance of smoke from a distance:\n\nCotinus, a common garden shrub.\nPsorothamnus spinosus, a Legume tree of the American Mojave desert. In the spring it has vibrant purple blooms.',
 "An apple is an edible fruit produced by an apple tree (Malus domestica). Apple trees are cultivated worldwide and are the most widely grown species in the genus Malus. The tree originated in Central Asia, where its wild ancestor, Malus sieversii, is still found today. Apples have been grown for thousands of years in Asia and Europe and were brought to

In [334]:
summaries = []
for s in species:
    try:
        summaries.append(wikipedia.summary(s + '(tree)'))
    except wikipedia.exceptions.DisambiguationError as e:
        print(e.options)
    

/home/ctchen/pyenvs/ds/lib/python3.8/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/ctchen/pyenvs/ds/lib/python3.8/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


["Hercules' Club (amulet)", 'Aralia spinosa', 'Zanthoxylum clava-herculis', 'Ojców National Park']
['Holly', 'Holly Tree Inn']
['Loblolly pine', 'Loblolly-bay', 'Loblolly boy', 'Loblolly Stable']


PageError: Page id "southwood tree" does not match any pages. Try another id!

In [56]:
# check leafsnap image for AL champion tree species.
leafimg = pd.read_csv('leafsnap-dataset-images.txt',sep='\t')
leafimg = leafimg.drop_duplicates('species').reset_index().drop('index',axis=1)
leafimg

,file_id,image_path,segmented_path,species,source
0,55497,dataset/images/lab/abies_concolor/ny1157-01-1.jpg,dataset/segmented/lab/abies_concolor/ny1157-01...,Abies concolor,lab
1,55697,dataset/images/lab/abies_nordmanniana/ny1057-0...,dataset/segmented/lab/abies_nordmanniana/ny105...,Abies nordmanniana,lab
2,55821,dataset/images/lab/acer_campestre/ny1079-01-1.jpg,dataset/segmented/lab/acer_campestre/ny1079-01...,Acer campestre,lab
3,69278,dataset/images/lab/acer_ginnala/ny1148-01-1.jpg,dataset/segmented/lab/acer_ginnala/ny1148-01-1...,Acer ginnala,lab
4,55924,dataset/images/lab/acer_griseum/ny1010-01-1.jpg,dataset/segmented/lab/acer_griseum/ny1010-01-1...,Acer griseum,lab
...,...,...,...,...,...
180,72681,dataset/images/lab/quercus_virginiana/wb1373-0...,dataset/segmented/lab/quercus_virginiana/wb137...,Quercus virginiana,lab
181,72921,dataset/images/lab/salix_caroliniana/wb1586-01...,dataset/segmented/lab/salix_caroliniana/wb1586...,Salix caroliniana,lab
182,73049,dataset/images/lab/salix_matsudana/wb1604-01-1...,dataset/segmented/lab/salix_matsudana/wb1604-0...,Salix matsudana,lab
183,73397,dataset/images/lab/staphylea_trifolia/pi0238-0...,dataset/segmented/lab/staphylea_trifolia/pi023...,Staphylea trifolia,lab


0               ilex vomitoria
1      liriodendron tulipifera
2          cladrastis kentukea
3             prunus umbellata
4           taxodium ascendens
                ...           
146          aesculus octandra
147        rhamnus caroliniana
148          bumelia lycioides
149         bumelia lanuginosa
150            juglans cinerea
Name: sci_lc, Length: 151, dtype: object

In [58]:
leafimg['sci_lc'] = [i.lower().strip() for i in leafimg.species.values]

In [59]:
merged = pd.merge(mdf, leafimg, on="sci_lc")

In [51]:
for i in mdf['sci_lc'].values:
    if 'acer' in i:
        print(i)

acer negundo,
acer leucoderme,
acer barbatum,
acer rubrum,
acer saccharinum,
acer saccharum,


In [54]:
for i in leafimg.sci_lc.values:
    if 'acer' in i :
        print(i)

acer campestre
acer ginnala
acer griseum
acer palmatum
acer platanoides
acer pseudoplatanus
acer rubrum
acer saccharinum
acer saccharum
platanus acerifolia
acer negundo
acer pensylvanicum


In [60]:
merged

,species_x,subspecies,scientific_name,year_named,circumference_in,height_in,spread_ft,total_points,county,year_remeasured,owner,nominator,sci_lc,file_id,image_path,segmented_path,species_y,source
0,YELLOW-POPLAR,N/A,Liriodendron tulipifera,1974.0,252.0,153.0,73.0,423.0,Lawrence,2016,U. S. Forest Service,W. E. Bustin,liriodendron tulipifera,60245,dataset/images/lab/liriodendron_tulipifera/pi0...,dataset/segmented/lab/liriodendron_tulipifera/...,Liriodendron tulipifera,lab
1,REDBUD,Eastern,Cercis canadensis,2000.0,86.0,36.0,34.0,131.0,Limestone,2017,Walter Hardiman,Doug Ezzell,cercis canadensis,58737,dataset/images/lab/cercis_canadensis/pi0001-01...,dataset/segmented/lab/cercis_canadensis/pi0001...,Cercis canadensis,lab
2,REDCEDAR,Eastern,Juniperus virginiana,1979.0,179.0,52.0,27.0,238.0,Wilcox,2017,Lou & John Harmon,William C. Carrigan,juniperus virginiana,58838,dataset/images/lab/juniperus_virginiana/pi1016...,dataset/segmented/lab/juniperus_virginiana/pi1...,Juniperus virginiana,lab
3,SASSAFRAS,N/A,Sassafras albidum,2020.0,162.0,47.0,57.0,223.0,Madison,2025,City of Huntsville,Marc Byers,sassafras albidum,4593,dataset/images/field/sassafras_albidum/1249061...,dataset/segmented/field/sassafras_albidum/1249...,Sassafras albidum,field
4,SASSAFRAS,N/A,Sassafras albidum,1992.0,140.0,74.0,33.0,222.0,Dallas,2017,Sarah Petty Dagenhart,Larry Brooks & Jason Shelton,sassafras albidum,4593,dataset/images/field/sassafras_albidum/1249061...,dataset/segmented/field/sassafras_albidum/1249...,Sassafras albidum,field
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,BEECH,American,Fagus grandifolia,2003.0,143.0,101.0,56.0,258.0,Autauga,2018,Cary McDaniel,Cary McDaniel,fagus grandifolia,56844,dataset/images/lab/fagus_grandifolia/wb1001-01...,dataset/segmented/lab/fagus_grandifolia/wb1001...,Fagus grandifolia,lab
68,BIRCH,River,Betula nigra,2014.0,100.0,92.0,73.0,210.0,Lee,2019,Michael Murray,Patrick Thompson,betula nigra,16019,dataset/images/field/betula_nigra/1258487222_0...,dataset/segmented/field/betula_nigra/125848722...,Betula nigra,field
69,BIRCH,Sweet,Betula lenta,2008.0,53.0,54.0,31.0,115.0,DeKalb,2018,DeSoto State Park,Jason Shelton,betula lenta,4375,dataset/images/field/betula_lenta/1248106459_0...,dataset/segmented/field/betula_lenta/124810645...,Betula lenta,field
70,BLACKGUM,N/A,Nyssa sylvatica,1988.0,128.0,108.0,49.0,248.0,Elmore,2017,Joseph Collins,Jill Browning,nyssa sylvatica,62140,dataset/images/lab/nyssa_sylvatica/ny1128-01-1...,dataset/segmented/lab/nyssa_sylvatica/ny1128-0...,Nyssa sylvatica,lab


In [47]:
grp = merged.groupby('sci_lc')
grp.species_x.count()

sci_lc
quercus shumardii    1
Name: species_x, dtype: int64

In [8]:
!java -version

openjdk version "11.0.10" 2021-01-19
OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.20.04)
OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.20.04, mixed mode, sharing)


In [9]:
!pip install -q tabula-py

In [10]:
import tabula

tabula.environment_info()

Python version:
    3.9.4 (default, Apr  9 2021, 16:34:09) 
[GCC 7.3.0]
Java version:
    openjdk version "11.0.10" 2021-01-19
OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.20.04)
OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.20.04, mixed mode, sharing)
tabula-py version: 2.2.0
platform: Linux-4.19.128-microsoft-standard-x86_64-with-glibc2.31
uname:
    uname_result(system='Linux', node='MSI', release='4.19.128-microsoft-standard', version='#1 SMP Tue Jun 23 12:58:10 UTC 2020', machine='x86_64')
linux_distribution: ('Ubuntu', '20.04', 'focal')
mac_ver: ('', ('', '', ''), '')
    
